## 5. Recurrent Neural Networks

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import json
import pandas as pd
file_path1 = '/content/drive/MyDrive/Colab Notebooks/train_datawithout3.json'
file_path2 = '/content/drive/MyDrive/Colab Notebooks/test_datawithout3.json'

reviews_train_lemmatize_split = json.load(open(file_path1))
reviews_test_lemmatize_split = json.load(open(file_path2))

file_path3 = '/content/drive/MyDrive/Colab Notebooks/train_data.json'
file_path4 = '/content/drive/MyDrive/Colab Notebooks/test_data.json'

reviews_train_lemmatize_split1 = json.load(open(file_path3))
reviews_test_lemmatize_split1 = json.load(open(file_path4))

y_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/bin_label_train.csv')
y_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/bin_label_test.csv')
y_train1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ter_label_train.csv')
y_test1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ter_label_test.csv')

### 5.1 Word Embedding

In [3]:
def word_embed_2(review, m):
    doc = []
    n = 0 
    for r in review:
        if r in m:
            n += 1
            doc.append(m[r])
            if n==50:
                break
    while n!=50:
        doc.append(np.zeros(300))
        n += 1
    return doc

Google Word2vec

In [4]:
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

In [7]:
import numpy as np
# deal with dataset without class 3
x_train_pretrain_bin_50 = []
x_test_pretrain_bin_50 = []
for review in reviews_train_lemmatize_split:
    x_train_pretrain_bin_50.append(word_embed_2(review, wv))
for review in reviews_test_lemmatize_split:
    x_test_pretrain_bin_50.append(word_embed_2(review, wv))

In [39]:
# deal with dataset with class 3
x_train_pretrain_ter_50 = []
x_test_pretrain_ter_50 = []
for review in reviews_train_lemmatize_split1:
    x_train_pretrain_ter_50.append(word_embed_2(review, wv))
    
for review in reviews_test_lemmatize_split1:
    x_test_pretrain_ter_50.append(word_embed_2(review, wv))

my word2vec model

In [5]:
import gensim
model = gensim.models.Word2Vec.load('/content/drive/MyDrive/Colab Notebooks/w2v')

In [27]:
x_train_my_bin_50 = []
x_test_my_bin_50 = []
for review in reviews_train_lemmatize_split:
    x_train_my_bin_50.append(word_embed_2(review, model.wv))
    
for review in reviews_test_lemmatize_split:
    x_test_my_bin_50.append(word_embed_2(review, model.wv))

In [7]:
# deal with dataset with class 3
import numpy as np
x_train_my_ter_50 = []
x_test_my_ter_50 = []
for review in reviews_train_lemmatize_split1:
    x_train_my_ter_50.append(word_embed_2(review, model.wv))
    
for review in reviews_test_lemmatize_split1:
    x_test_my_ter_50.append(word_embed_2(review, model.wv))

### 5.2 DataLoader create

In [9]:
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence

In [10]:
class Reviews(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __len__(self):
        return len(self.y)
    def __getitem__(self, index):
        embeds = self.x[index]
        label = self.y[index]
        
        return embeds, label

In [13]:
binary_train_y = np.squeeze(y_train.values)
binary_test_y = np.squeeze(y_test.values)
binary_train_RNN = Reviews(x_train_pretrain_bin_50, binary_train_y)
binary_test_RNN = Reviews(x_test_pretrain_bin_50, binary_test_y)

In [11]:

def pad_data(batch):
    X = [torch.tensor(x[0], dtype=torch.float32) for x in batch]
    Y = [x[1] for x in batch]
    X_len = [len(x[0]) for x in batch]
    X_pad = pad_sequence(X, batch_first=True, padding_value=0)
    return X_pad, Y, X_len

In [ ]:

num_workers = 0
batch_size = 64
valid_size = 0.2
num_train = len(x_train_pretrain_bin_50)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size*num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# sample the train and validation batch
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)
# DataLoaders 
binary_train_dataloader_RNN = DataLoader(binary_train_RNN, sampler=train_sampler, batch_size=batch_size,collate_fn=pad_data)
binary_valid_dataloader_RNN = DataLoader(binary_train_RNN, sampler=valid_sampler, batch_size=batch_size,collate_fn=pad_data)
binary_test_dataloader_RNN = DataLoader(binary_test_RNN, batch_size=batch_size,collate_fn=pad_data)

In [45]:
# deal with dataset with class 3 use Google word2vec
ternary_train_y = np.squeeze(y_train1.values)
ternary_train_y[ternary_train_y==3]=0
ternary_test_y = np.squeeze(y_test1.values)
ternary_train_y[ternary_train_y==3]=0
ternary_train_RNN = Reviews(x_train_pretrain_ter_50, ternary_train_y)
ternary_test_RNN = Reviews(x_test_pretrain_ter_50, ternary_test_y)

num_workers = 0
batch_size = 64
valid_size = 0.2
num_train = len(x_train_pretrain_ter_50)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size*num_train))
train_idx, valid_idx = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

ternary_train_dataloader_RNN = DataLoader(ternary_train_RNN, sampler=train_sampler, batch_size=batch_size,collate_fn=pad_data)
ternary_valid_dataloader_RNN = DataLoader(ternary_train_RNN, sampler=valid_sampler, batch_size=batch_size,collate_fn=pad_data)
ternary_test_dataloader_RNN = DataLoader(ternary_test_RNN, batch_size=batch_size,collate_fn=pad_data)

## 5.3 Model

### (a) RNN

#### (a)-1 Binary Classification

Google Word2vec

In [14]:
import torch.nn as nn
import torch
import torch.nn.functional as F
from tqdm import tqdm
from torch.nn.utils.rnn import pack_padded_sequence
from torch.nn.utils.rnn import pad_packed_sequence

In [15]:
input_size = 300
hidden_size = 50
num_layers = 1
sequence_length = 50
output_size = 2

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size * sequence_length, output_size)

    def forward(self, x, lengths):
        # initize the hidden layer
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        x_packed = pack_padded_sequence(x, lengths, batch_first=True, enforce_sorted = False)
        out, _ = self.rnn(x_packed, h0)
        out, out_lengths = pad_packed_sequence(out, batch_first=True)
        out = out.reshape(out.shape[0],-1)
        output = self.fc(out)
        return output

In [ ]:
rnn_bi_1 = RNN(input_size, hidden_size, num_layers, output_size)
crite = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn_bi_1.parameters(), lr=0.001)

In [ ]:
valid_loss_min = np.Inf
num_epochs = 5
for epoch in range(num_epochs):
  train_loss = 0.0
  valid_loss = 0.0
  rnn_bi_1.train()
  for batch_idx, (data, targets, lengths) in enumerate(tqdm(binary_train_dataloader_RNN)):
    targets = torch.tensor(targets, dtype=torch.long)
    scores = rnn_bi_1(data, lengths)
    loss = crite(scores, targets)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    train_loss += loss.item()*data.size(0)
  rnn_bi_1.eval()
  for batch_idx, (data, targets, lengths) in enumerate(tqdm(binary_train_dataloader_RNN)):
    targets = torch.tensor(targets, dtype=torch.long)
    scores = rnn_bi_1(data, lengths)
    loss = crite(scores, targets)
    valid_loss += loss.item()*data.size(0)

  train_loss = train_loss/(num_train - split)
  valid_loss = valid_loss/split
  print('Epoch: {} Training Loss: {:.6f}, Validation Loss: {:.6f}\n'.format(
      epoch+1, train_loss, valid_loss))


100%|██████████| 2000/2000 [03:38<00:00,  9.15it/s]


Epoch: 1 Training Loss: 0.431976, Validation Loss: 1.553999


100%|██████████| 2000/2000 [03:37<00:00,  9.18it/s]


Epoch: 2 Training Loss: 0.390333, Validation Loss: 1.584250


100%|██████████| 2000/2000 [03:35<00:00,  9.30it/s]


Epoch: 3 Training Loss: 0.368653, Validation Loss: 1.402450


100%|██████████| 2000/2000 [03:36<00:00,  9.25it/s]


Epoch: 4 Training Loss: 0.349889, Validation Loss: 1.317528


100%|██████████| 2000/2000 [03:39<00:00,  9.12it/s]

Epoch: 5 Training Loss: 0.338538, Validation Loss: 1.249719


In [18]:
def test_acc(loader, model):
  corr_cnt = 0
  sampe_cnt = 0
  model.eval()
  with torch.no_grad():
    for data, targets, lengths in loader:
      scores = model(data, lengths)
      _, pred = scores.max(1)
      corr_cnt += sum(np.array(pred)==targets)
      sampe_cnt += pred.size(0)
  model.train()
  return corr_cnt/sampe_cnt

In [ ]:
print("accuracy of binary test set by using google word2vec:", test_acc(binary_test_dataloader_RNN,rnn_bi_1))

accuracy of binary test set by using google word2vec: 0.8449


my word2vec

In [29]:
binary_train_y = np.squeeze(y_train.values)
binary_test_y = np.squeeze(y_test.values)
binary_train_RNN = Reviews(x_train_my_bin_50, binary_train_y)
binary_test_RNN = Reviews(x_test_my_bin_50, binary_test_y)

num_workers = 0
batch_size = 64
valid_size = 0.2
num_train = len(x_train_my_bin_50)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size*num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# sample the train and validation batch
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# DataLoaders 
binary_train_dataloader_RNN = DataLoader(binary_train_RNN, sampler=train_sampler, batch_size=batch_size,collate_fn=pad_data)
binary_valid_dataloader_RNN = DataLoader(binary_train_RNN, sampler=valid_sampler, batch_size=batch_size,collate_fn=pad_data)
binary_test_dataloader_RNN = DataLoader(binary_test_RNN, batch_size=batch_size,collate_fn=pad_data)

In [30]:
rnn_bi_2 = RNN(input_size, hidden_size, num_layers, output_size)
crite = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn_bi_2.parameters(), lr=0.001)

In [31]:
valid_loss_min = np.Inf
num_epochs = 5
for epoch in range(num_epochs):
  train_loss = 0.0
  valid_loss = 0.0
  rnn_bi_2.train()
  for batch_idx, (data, targets, lengths) in enumerate(tqdm(binary_train_dataloader_RNN)):
    targets = torch.tensor(targets, dtype=torch.long)
    scores = rnn_bi_2(data, lengths)
    loss = crite(scores, targets)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    train_loss += loss.item()*data.size(0)
  rnn_bi_2.eval()
  for batch_idx, (data, targets, lengths) in enumerate(tqdm(binary_train_dataloader_RNN)):
    targets = torch.tensor(targets, dtype=torch.long)
    scores = rnn_bi_2(data, lengths)
    loss = crite(scores, targets)
    valid_loss += loss.item()*data.size(0)

  train_loss = train_loss/(num_train - split)
  valid_loss = valid_loss/split
  print('Epoch: {} Training Loss: {:.6f}, Validation Loss: {:.6f}\n'.format(
      epoch+1, train_loss, valid_loss))


100%|██████████| 2000/2000 [03:29<00:00,  9.53it/s]


Epoch: 1 Training Loss: 0.383375, Validation Loss: 1.341032



100%|██████████| 2000/2000 [03:45<00:00,  8.85it/s]


Epoch: 2 Training Loss: 0.353551, Validation Loss: 1.322925



100%|██████████| 2000/2000 [03:32<00:00,  9.42it/s]


Epoch: 3 Training Loss: 0.340988, Validation Loss: 1.238473



100%|██████████| 2000/2000 [03:30<00:00,  9.52it/s]


Epoch: 4 Training Loss: 0.329981, Validation Loss: 1.185232



100%|██████████| 2000/2000 [03:31<00:00,  9.46it/s]

Epoch: 5 Training Loss: 0.321567, Validation Loss: 1.216971



In [32]:
print("accuracy of binary RNN by using my word2vec model:", test_acc(binary_test_dataloader_RNN,rnn_bi_2))

accuracy of binary RNN by using my word2vec model: 0.851


#### (a)-2 Ternary Classification

Google Word2vec

In [47]:
rnn_ter_1 = RNN(input_size, hidden_size, num_layers, 3)
crite = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn_ter_1.parameters(), lr=0.001)
valid_loss_min = np.Inf
num_epochs = 5
for epoch in range(num_epochs):
  train_loss = 0.0
  valid_loss = 0.0
  rnn_ter_1.train()
  for batch_idx, (data, targets, lengths) in enumerate(tqdm(ternary_train_dataloader_RNN)):
    targets = torch.tensor(targets, dtype=torch.long)
    scores = rnn_ter_1(data, lengths)
    loss = crite(scores, targets)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    train_loss += loss.item()*data.size(0)
  rnn_ter_1.eval()
  for batch_idx, (data, targets, lengths) in enumerate(tqdm(ternary_train_dataloader_RNN)):
    targets = torch.tensor(targets, dtype=torch.long)
    scores = rnn_ter_1(data, lengths)
    loss = crite(scores, targets)
    valid_loss += loss.item()*data.size(0)

  train_loss = train_loss/(num_train - split)
  valid_loss = valid_loss/split
  print('Epoch: {} Training Loss: {:.6f}, Validation Loss: {:.6f}\n'.format(
      epoch+1, train_loss, valid_loss))


100%|██████████| 2500/2500 [04:22<00:00,  9.53it/s]


Epoch: 1 Training Loss: 0.823216, Validation Loss: 3.090475



100%|██████████| 2500/2500 [04:25<00:00,  9.43it/s]


Epoch: 2 Training Loss: 0.781442, Validation Loss: 2.997661



100%|██████████| 2500/2500 [04:22<00:00,  9.51it/s]


Epoch: 3 Training Loss: 0.765494, Validation Loss: 2.921576



100%|██████████| 2500/2500 [04:23<00:00,  9.50it/s]


Epoch: 4 Training Loss: 0.749599, Validation Loss: 2.852113



100%|██████████| 2500/2500 [04:25<00:00,  9.40it/s]

Epoch: 5 Training Loss: 0.738646, Validation Loss: 2.807690



In [49]:
print("accuracy of ternary RNN by using Google word2vec:", test_acc(ternary_test_dataloader_RNN,rnn_ter_1))

accuracy of ternary RNN by using Google word2vec: 0.63382


my word2vec

In [12]:
# deal with dataset with class 3
ternary_train_y = np.squeeze(y_train1.values)
ternary_train_y[ternary_train_y==3]=0
ternary_test_y = np.squeeze(y_test1.values)
ternary_train_y[ternary_train_y==3]=0
ternary_train_RNN = Reviews(x_train_my_ter_50, ternary_train_y)
ternary_test_RNN = Reviews(x_test_my_ter_50, ternary_test_y)

num_workers = 0
batch_size = 64
valid_size = 0.2
num_train = len(x_train_my_ter_50)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size*num_train))
train_idx, valid_idx = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

ternary_train_dataloader_RNN = DataLoader(ternary_train_RNN, sampler=train_sampler, batch_size=batch_size,collate_fn=pad_data)
ternary_valid_dataloader_RNN = DataLoader(ternary_train_RNN, sampler=valid_sampler, batch_size=batch_size,collate_fn=pad_data)
ternary_test_dataloader_RNN = DataLoader(ternary_test_RNN, batch_size=batch_size,collate_fn=pad_data)

In [16]:
rnn_ter_2 = RNN(input_size, hidden_size, num_layers, 3)
crite = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn_ter_2.parameters(), lr=0.001)
valid_loss_min = np.Inf
num_epochs = 5
for epoch in range(num_epochs):
  train_loss = 0.0
  valid_loss = 0.0
  rnn_ter_2.train()
  for batch_idx, (data, targets, lengths) in enumerate(tqdm(ternary_train_dataloader_RNN)):
    targets = torch.tensor(targets, dtype=torch.long)
    scores = rnn_ter_2(data, lengths)
    loss = crite(scores, targets)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    train_loss += loss.item()*data.size(0)
  rnn_ter_2.eval()
  for batch_idx, (data, targets, lengths) in enumerate(tqdm(ternary_train_dataloader_RNN)):
    targets = torch.tensor(targets, dtype=torch.long)
    scores = rnn_ter_2(data, lengths)
    loss = crite(scores, targets)
    valid_loss += loss.item()*data.size(0)

  train_loss = train_loss/(num_train - split)
  valid_loss = valid_loss/split
  print('Epoch: {} Training Loss: {:.6f}, Validation Loss: {:.6f}\n'.format(
      epoch+1, train_loss, valid_loss))


100%|██████████| 2500/2500 [04:24<00:00,  9.46it/s]


Epoch: 1 Training Loss: 0.785248, Validation Loss: 2.885084



100%|██████████| 2500/2500 [04:24<00:00,  9.44it/s]


Epoch: 2 Training Loss: 0.751025, Validation Loss: 2.837594



100%|██████████| 2500/2500 [04:23<00:00,  9.49it/s]


Epoch: 3 Training Loss: 0.736676, Validation Loss: 2.782840



100%|██████████| 2500/2500 [04:23<00:00,  9.47it/s]


Epoch: 4 Training Loss: 0.727402, Validation Loss: 2.755624



100%|██████████| 2500/2500 [04:25<00:00,  9.41it/s]

Epoch: 5 Training Loss: 0.717901, Validation Loss: 2.719879



In [19]:
print("accuracy of ternary RNN by using my word2vec:", test_acc(ternary_test_dataloader_RNN,rnn_ter_2))

accuracy of ternary RNN by using my word2vec: 0.63576


### (b) GRU

#### (b)-1 Binary Classification

Google Word2vec

In [16]:
num_workers = 0
batch_size = 128
valid_size = 0.2
num_train = len(x_train_pretrain_bin_50)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size*num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# sample the train and validation batch
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)
# DataLoaders 
binary_train_dataloader_RNN = DataLoader(binary_train_RNN, sampler=train_sampler, batch_size=batch_size,collate_fn=pad_data)
binary_valid_dataloader_RNN = DataLoader(binary_train_RNN, sampler=valid_sampler, batch_size=batch_size,collate_fn=pad_data)
binary_test_dataloader_RNN = DataLoader(binary_test_RNN, batch_size=batch_size,collate_fn=pad_data)

In [20]:
input_size = 300
hidden_size = 50
num_layers = 1
batch_size = 64
sequence_length = 50
output_size = 2

class GRU(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(GRU, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size * sequence_length, output_size)

    def forward(self, x, lengths):
        # initize the hidden layer
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        x_packed = pack_padded_sequence(x, lengths, batch_first=True, enforce_sorted = False)
        out, _ = self.gru(x_packed, h0)
        out, out_lengths = pad_packed_sequence(out, batch_first=True)
        out = out.reshape(out.shape[0],-1)
        output = self.fc(out)
        return output

In [18]:
gru_bi_1 = GRU(input_size, hidden_size, num_layers, output_size)
crite = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(gru_bi_1.parameters(), lr=0.001)

valid_loss_min = np.Inf
num_epochs = 5
for epoch in range(num_epochs):
  train_loss = 0.0
  valid_loss = 0.0
  gru_bi_1.train()
  for batch_idx, (data, targets, lengths) in enumerate(tqdm(binary_train_dataloader_RNN)):
    targets = torch.tensor(targets, dtype=torch.long)
    scores = gru_bi_1(data, lengths)
    loss = crite(scores, targets)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    train_loss += loss.item()*data.size(0)
  gru_bi_1.eval()
  for batch_idx, (data, targets, lengths) in enumerate(tqdm(binary_train_dataloader_RNN)):
    targets = torch.tensor(targets, dtype=torch.long)
    scores = gru_bi_1(data, lengths)
    loss = crite(scores, targets)
    valid_loss += loss.item()*data.size(0)

  train_loss = train_loss/(num_train - split)
  valid_loss = valid_loss/split
  print('Epoch: {} Training Loss: {:.6f}, Validation Loss: {:.6f}\n'.format(
      epoch+1, train_loss, valid_loss))


100%|██████████| 1000/1000 [03:36<00:00,  4.61it/s]


Epoch: 1 Training Loss: 0.386307, Validation Loss: 1.337222



100%|██████████| 1000/1000 [03:37<00:00,  4.61it/s]


Epoch: 2 Training Loss: 0.315908, Validation Loss: 1.235545



100%|██████████| 1000/1000 [03:37<00:00,  4.60it/s]


Epoch: 3 Training Loss: 0.296045, Validation Loss: 1.101958



100%|██████████| 1000/1000 [03:35<00:00,  4.63it/s]


Epoch: 4 Training Loss: 0.279506, Validation Loss: 1.033319



100%|██████████| 1000/1000 [03:37<00:00,  4.60it/s]

Epoch: 5 Training Loss: 0.267655, Validation Loss: 0.997901



In [22]:
print("accuracy of binary GRU by using google word2vec:", test_acc(binary_test_dataloader_RNN,gru_bi_1))

accuracy of binary GRU by using google word2vec: 0.8768


my word2vec binary

In [33]:
num_workers = 0
batch_size = 128
valid_size = 0.2
num_train = len(x_train_my_bin_50)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size*num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# sample the train and validation batch
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)
# DataLoaders 
binary_train_dataloader_RNN = DataLoader(binary_train_RNN, sampler=train_sampler, batch_size=batch_size,collate_fn=pad_data)
binary_valid_dataloader_RNN = DataLoader(binary_train_RNN, sampler=valid_sampler, batch_size=batch_size,collate_fn=pad_data)
binary_test_dataloader_RNN = DataLoader(binary_test_RNN, batch_size=batch_size,collate_fn=pad_data)

In [34]:
gru_bi_2 = GRU(input_size, hidden_size, num_layers, output_size)
crite = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(gru_bi_2.parameters(), lr=0.001)

valid_loss_min = np.Inf
num_epochs = 5
for epoch in range(num_epochs):
  train_loss = 0.0
  valid_loss = 0.0
  gru_bi_2.train()
  for batch_idx, (data, targets, lengths) in enumerate(tqdm(binary_train_dataloader_RNN)):
    targets = torch.tensor(targets, dtype=torch.long)
    scores = gru_bi_2(data, lengths)
    loss = crite(scores, targets)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    train_loss += loss.item()*data.size(0)
  gru_bi_2.eval()
  for batch_idx, (data, targets, lengths) in enumerate(tqdm(binary_train_dataloader_RNN)):
    targets = torch.tensor(targets, dtype=torch.long)
    scores = gru_bi_2(data, lengths)
    loss = crite(scores, targets)
    valid_loss += loss.item()*data.size(0)

  train_loss = train_loss/(num_train - split)
  valid_loss = valid_loss/split
  print('Epoch: {} Training Loss: {:.6f}, Validation Loss: {:.6f}\n'.format(
      epoch+1, train_loss, valid_loss))


100%|██████████| 1000/1000 [03:34<00:00,  4.66it/s]


Epoch: 1 Training Loss: 0.345819, Validation Loss: 1.165177



100%|██████████| 1000/1000 [03:34<00:00,  4.66it/s]


Epoch: 2 Training Loss: 0.298722, Validation Loss: 1.066820



100%|██████████| 1000/1000 [03:36<00:00,  4.62it/s]


Epoch: 3 Training Loss: 0.277815, Validation Loss: 0.966617



100%|██████████| 1000/1000 [03:35<00:00,  4.64it/s]


Epoch: 4 Training Loss: 0.260743, Validation Loss: 0.897642



100%|██████████| 1000/1000 [03:37<00:00,  4.60it/s]

Epoch: 5 Training Loss: 0.245677, Validation Loss: 0.854250



In [35]:
print("accuracy of binary GRU by using my word2vec model:", test_acc(binary_test_dataloader_RNN,gru_bi_2))

accuracy of binary GRU by using my word2vec model: 0.86945


#### (b)-2 Ternary Classification

Google Word2Vec

In [52]:
gru_ter_1 = GRU(input_size, hidden_size, num_layers, 3)
crite = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(gru_ter_1.parameters(), lr=0.001)

valid_loss_min = np.Inf
num_epochs = 5
for epoch in range(num_epochs):
  train_loss = 0.0
  valid_loss = 0.0
  gru_ter_1.train()
  for batch_idx, (data, targets, lengths) in enumerate(tqdm(ternary_train_dataloader_RNN)):
    targets = torch.tensor(targets, dtype=torch.long)
    scores = gru_ter_1(data, lengths)
    loss = crite(scores, targets)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    train_loss += loss.item()*data.size(0)
  gru_ter_1.eval()
  for batch_idx, (data, targets, lengths) in enumerate(tqdm(ternary_train_dataloader_RNN)):
    targets = torch.tensor(targets, dtype=torch.long)
    scores = gru_ter_1(data, lengths)
    loss = crite(scores, targets)
    valid_loss += loss.item()*data.size(0)

  train_loss = train_loss/(num_train - split)
  valid_loss = valid_loss/split
  print('Epoch: {} Training Loss: {:.6f}, Validation Loss: {:.6f}\n'.format(
      epoch+1, train_loss, valid_loss))


100%|██████████| 2500/2500 [04:45<00:00,  8.77it/s]


Epoch: 1 Training Loss: 0.752938, Validation Loss: 2.762881



100%|██████████| 2500/2500 [04:38<00:00,  8.97it/s]


Epoch: 2 Training Loss: 0.684669, Validation Loss: 2.650215



100%|██████████| 2500/2500 [04:43<00:00,  8.82it/s]


Epoch: 3 Training Loss: 0.661130, Validation Loss: 2.600601



100%|██████████| 2500/2500 [04:55<00:00,  8.47it/s]


Epoch: 4 Training Loss: 0.642333, Validation Loss: 2.440293



100%|██████████| 2500/2500 [04:39<00:00,  8.94it/s]

Epoch: 5 Training Loss: 0.626274, Validation Loss: 2.409096



In [53]:
print("accuracy of ternary GRU by using Google word2vec:", test_acc(ternary_test_dataloader_RNN,gru_ter_1))

accuracy of ternary GRU by using Google word2vec: 0.6524


my word2vec

In [21]:
gru_ter_2 = GRU(input_size, hidden_size, num_layers, 3)
crite = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(gru_ter_2.parameters(), lr=0.001)

valid_loss_min = np.Inf
num_epochs = 5
for epoch in range(num_epochs):
  train_loss = 0.0
  valid_loss = 0.0
  gru_ter_2.train()
  for batch_idx, (data, targets, lengths) in enumerate(tqdm(ternary_train_dataloader_RNN)):
    targets = torch.tensor(targets, dtype=torch.long)
    scores = gru_ter_2(data, lengths)
    loss = crite(scores, targets)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    train_loss += loss.item()*data.size(0)
  gru_ter_2.eval()
  for batch_idx, (data, targets, lengths) in enumerate(tqdm(ternary_train_dataloader_RNN)):
    targets = torch.tensor(targets, dtype=torch.long)
    scores = gru_ter_2(data, lengths)
    loss = crite(scores, targets)
    valid_loss += loss.item()*data.size(0)

  train_loss = train_loss/(num_train - split)
  valid_loss = valid_loss/split
  print('Epoch: {} Training Loss: {:.6f}, Validation Loss: {:.6f}\n'.format(
      epoch+1, train_loss, valid_loss))


100%|██████████| 2500/2500 [04:36<00:00,  9.03it/s]


Epoch: 1 Training Loss: 0.739198, Validation Loss: 2.715519



100%|██████████| 2500/2500 [04:38<00:00,  8.96it/s]


Epoch: 2 Training Loss: 0.694046, Validation Loss: 2.580087



100%|██████████| 2500/2500 [04:39<00:00,  8.95it/s]


Epoch: 3 Training Loss: 0.674032, Validation Loss: 2.543447



100%|██████████| 2500/2500 [04:36<00:00,  9.04it/s]


Epoch: 4 Training Loss: 0.658359, Validation Loss: 2.456954



100%|██████████| 2500/2500 [04:34<00:00,  9.12it/s]

Epoch: 5 Training Loss: 0.645589, Validation Loss: 2.398699



In [22]:
print("accuracy of ternary GRU by using my word2vec model:", test_acc(ternary_test_dataloader_RNN,gru_ter_2))

accuracy of ternary GRU by using my word2vec model: 0.64278


The accuracy of binary RNN using Google word2vec is <font color=red>0.8449</font>

The accuracy of binary RNN is using my word2vec model is <font color=red>0.851</font>

The accuracy of ternary RNN is using Google word2vec is <font color=red>0.63382</font>

The accuracy of ternary RNN using my word2vec model is <font color=red>0.63576</font>

The accuracy of binary GRU using Google word2vec is <font color=red>0.8768</font>

The accuracy of binary GRU is using my word2vec model is <font color=red>0.86945</font>

The accuracy of ternary GRU is using Google word2vec is <font color=red>0.6524</font>

The accuracy of ternary GRU using my word2vec model is <font color=red>0.64278</font>


Samiliar with part 4, the accuracies of binary classification are higher than ternary classification. Besides, the overall accuracy tested by RNN is higher than the performance of FNN.